In [148]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('./DDos-dataset/DDos.csv') 
print(data.count())

columns_to_keep = [
    'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 
    'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Fwd IAT Mean', 
    'Bwd IAT Mean', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 
    'Down/Up Ratio', 'Average Packet Size', 'Subflow Fwd Packets', 
    'Subflow Bwd Packets', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 
    'Idle Mean', 'Idle Std', 'Label'
]
data = data[columns_to_keep]

Flow ID             225745
Source IP           225745
Source Port         225745
Destination IP      225745
Destination Port    225745
                     ...  
Idle Mean           225745
Idle Std            225745
Idle Max            225745
Idle Min            225745
Label               225745
Length: 85, dtype: int64


In [149]:
data['Label'] = data['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)

data.replace([np.inf, -np.inf], np.nan, inplace=True)

data.fillna(data.mean(), inplace=True)


X = data.drop(columns=['Label']).values
y = data['Label'].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [123]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax')) 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [125]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, 
                    validation_data=(X_val, y_val), verbose=1)

Epoch 1/20
4939/4939 [==============================] - 18s 3ms/step - loss: 0.1405 - accuracy: 0.9488 - val_loss: 0.0745 - val_accuracy: 0.9668
Epoch 2/20
4939/4939 [==============================] - 15s 3ms/step - loss: 0.0545 - accuracy: 0.9833 - val_loss: 0.0330 - val_accuracy: 0.9882
Epoch 3/20
4939/4939 [==============================] - 15s 3ms/step - loss: 0.0342 - accuracy: 0.9886 - val_loss: 0.0282 - val_accuracy: 0.9889
Epoch 4/20
4939/4939 [==============================] - 15s 3ms/step - loss: 0.0287 - accuracy: 0.9903 - val_loss: 0.0243 - val_accuracy: 0.9938
Epoch 5/20
4939/4939 [==============================] - 16s 3ms/step - loss: 0.0245 - accuracy: 0.9925 - val_loss: 0.0202 - val_accuracy: 0.9950
Epoch 6/20
4939/4939 [==============================] - 16s 3ms/step - loss: 0.0198 - accuracy: 0.9944 - val_loss: 0.0155 - val_accuracy: 0.9959
Epoch 7/20
4939/4939 [==============================] - 16s 3ms/step - loss: 0.0162 - accuracy: 0.9958 - val_loss: 0.0127 - val_ac

In [127]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")

1059/1059 [==============================] - 2s 2ms/step - loss: 0.0064 - accuracy: 0.9986
Test Accuracy: 1.00


In [129]:
attack_example = np.array([[30000, 100, 50, 1.2, 2.5, 1000, 900, 800, 1, 1, 0, 0.9, 1500, 75, 50, 10000, 10000, 500, 10]])
attack_example = scaler.transform(attack_example)
attack_example = attack_example.reshape((1, 1, attack_example.shape[1]))

normal_example = np.array([[5000, 20, 15, 0.5, 1.0, 200, 150, 100, 0, 1, 0, 0.5, 600, 20, 15, 5000, 5000, 100, 5]])
normal_example = scaler.transform(normal_example)
normal_example = normal_example.reshape((1, 1, normal_example.shape[1]))

attack_pred = model.predict(attack_example)
normal_pred = model.predict(normal_example)

print("Prediction pour l'attaque DDoS:", "Attaque" if np.argmax(attack_pred) == 1 else "Non-attaque")
print("Prediction pour la session normale:", "Attaque" if np.argmax(normal_pred) == 1 else "Non-attaque")

1/1 [==============================] - 0s 21ms/step
Prediction pour l'attaque DDoS: Non-attaque
Prediction pour la session normale: Non-attaque
